In [40]:
import pandas as pd
movies=pd.read_csv('movies_with_cat_new.csv')

In [41]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base",
                      top_k=None,
                      device=0)
classifier("I love this!")

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528688922524452},
  {'label': 'neutral', 'score': 0.005764586851000786},
  {'label': 'anger', 'score': 0.004419783595949411},
  {'label': 'sadness', 'score': 0.002092392183840275},
  {'label': 'disgust', 'score': 0.0016119893407449126},
  {'label': 'fear', 'score': 0.0004138524236623198}]]

In [42]:
classifier(movies['description'][0])

[[{'label': 'fear', 'score': 0.9261465072631836},
  {'label': 'anger', 'score': 0.024331700056791306},
  {'label': 'neutral', 'score': 0.02144649624824524},
  {'label': 'sadness', 'score': 0.01804129220545292},
  {'label': 'disgust', 'score': 0.006444478873163462},
  {'label': 'surprise', 'score': 0.0026720319874584675},
  {'label': 'joy', 'score': 0.0009175881859846413}]]

In [43]:
movies['description'][0]

'Peter Parker is unmasked and no longer able to separate his normal life from the high-stakes of being a super-hero. When he asks for help from Doctor Strange the stakes become even more dangerous, forcing him to discover what it truly means to be Spider-Man.'

In [44]:
import numpy as np
emotion_labels=['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
ids=[]
emotion_scores={label:[] for label in emotion_labels}


def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x['label'])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]['score'])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}


In [45]:
for i in range(20):
    ids.append(movies['movie_id'][i])
    sentences = movies['description'][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [46]:
emotion_scores

{'anger': [np.float64(0.07944381982088089),
  np.float64(0.1881798803806305),
  np.float64(0.06413359194993973),
  np.float64(0.06413359194993973),
  np.float64(0.24244974553585052),
  np.float64(0.13221636414527893),
  np.float64(0.1261821687221527),
  np.float64(0.7532293200492859),
  np.float64(0.06413359194993973),
  np.float64(0.3216216564178467),
  np.float64(0.06413359194993973),
  np.float64(0.15098333358764648),
  np.float64(0.20045159757137299),
  np.float64(0.06413359194993973),
  np.float64(0.0767819806933403),
  np.float64(0.06413359194993973),
  np.float64(0.09325892478227615),
  np.float64(0.24757660925388336),
  np.float64(0.06413359194993973),
  np.float64(0.5056110620498657)],
 'disgust': [np.float64(0.10400661826133728),
  np.float64(0.4141789376735687),
  np.float64(0.10400661826133728),
  np.float64(0.10400661826133728),
  np.float64(0.2062649428844452),
  np.float64(0.10400661826133728),
  np.float64(0.16332010924816132),
  np.float64(0.21924985945224762),
  np.fl

In [47]:
from tqdm import tqdm

emotion_labels=['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
ids=[]
emotion_scores={label:[] for label in emotion_labels}

for i in tqdm(range(len(movies))):
    ids.append(movies['movie_id'][i])
    sentences = movies['description'][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 7933/7933 [09:09<00:00, 14.44it/s]


In [48]:
import pandas as pd
emotion_df=pd.DataFrame(emotion_scores)
emotion_df['movie_id']=ids

In [49]:
emotion_df.head()

,anger,disgust,fear,joy,neutral,sadness,surprise,movie_id
0,0.079444,0.104007,0.892590,0.040564,0.549477,0.269045,0.078765,1
1,0.188180,0.414179,0.143998,0.040564,0.549477,0.111690,0.078765,2
2,0.064134,0.104007,0.987016,0.040564,0.549477,0.111690,0.078765,3
3,0.064134,0.104007,0.957093,0.876721,0.698927,0.111690,0.155995,4
4,0.242450,0.206265,0.139706,0.040564,0.549477,0.111690,0.078765,5


In [50]:
movies=pd.merge(movies,emotion_df,on='movie_id')

In [54]:
movies.head()

,release_year,title,description,Popularity,Vote_Count,rating,Original_Language,listed_in,Poster_Url,movie_id,tagged_des,anger,disgust,fear,joy,neutral,sadness,surprise
0,2021-12-15,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,5083.954,8940,8.3,en,"Action, Adventure, Science Fiction",https://image.tmdb.org/t/p/original/1g0dhYtq4i...,1,1 Peter Parker is unmasked and no longer able ...,0.079444,0.104007,0.892590,0.040564,0.549477,0.269045,0.078765
1,2022-03-01,The Batman,"In his second year of fighting crime, Batman u...",3827.658,1151,8.1,en,"Crime, Mystery, Thriller",https://image.tmdb.org/t/p/original/74xTEgt7R3...,2,"2 In his second year of fighting crime, Batman...",0.188180,0.414179,0.143998,0.040564,0.549477,0.111690,0.078765
2,2022-02-25,No Exit,Stranded at a rest stop in the mountains durin...,2618.087,122,6.3,en,Thriller,https://image.tmdb.org/t/p/original/vDHsLnOWKl...,3,3 Stranded at a rest stop in the mountains dur...,0.064134,0.104007,0.987016,0.040564,0.549477,0.111690,0.078765
3,2021-11-24,Encanto,"The tale of an extraordinary family, the Madri...",2402.201,5076,7.7,en,"Animation, Comedy, Family, Fantasy",https://image.tmdb.org/t/p/original/4j0PNHkMr5...,4,"4 The tale of an extraordinary family, the Mad...",0.064134,0.104007,0.957093,0.876721,0.698927,0.111690,0.155995
4,2021-12-22,The King's Man,As a collection of history's worst tyrants and...,1895.511,1793,7.0,en,"Action, Adventure, Thriller, War",https://image.tmdb.org/t/p/original/aq4Pwv5Xeu...,5,5 As a collection of history's worst tyrants a...,0.242450,0.206265,0.139706,0.040564,0.549477,0.111690,0.078765


In [52]:
movies['Poster_Url'].value_counts()[movies['Poster_Url'].value_counts() > 2]


Series([], Name: count, dtype: int64)

In [53]:
movies.to_csv('movies_with_emotion_new.csv', index=False)